# Part 1: Make your own container from base image

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. 

This code looks for an ECR repository in the account you're using and the current region. If the repository doesn't exist, the script will create it.

### Create Dockerfile

In [ ]:
base_image = "111016121260.dkr.ecr.us-east-2.amazonaws.com/ubuntu-16.04-base-with-pip"
%env base_image=$base_image

#create Dockerfile with base image
!echo FROM $base_image > container_rule/Dockerfile
!echo >> container_rule/Dockerfile
!echo "ENV PYTHONUNBUFFERED=TRUE" >> container_rule/Dockerfile
!echo "ENV PYTHONDONTWRITEBYTECODE=TRUE" >> container_rule/Dockerfile
!echo "ENV PATH=\"/opt/program:\${PATH}\"" >> container_rule/Dockerfile
!echo >> container_rule/Dockerfile
!echo "COPY code" /opt/program >> container_rule/Dockerfile
!echo "WORKDIR /opt/program" >> container_rule/Dockerfile


### Algorithm name as environment viariable

In [ ]:
algorithm_name="python-rule-based-1"
%env algorithm_name=$algorithm_name

### Get account info for Dockerfile

In [ ]:
get_account=!aws sts get-caller-identity --query Account --output text --endpoint-url https://sts.us-east-2.amazonaws.com
account = get_account[0]
%env account=$account

get_region=!aws configure get region
region = get_region[0]
%env region=$region

fullname=f"{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:latest"
%env fullname=$fullname

### Set permission for folders

In [ ]:
!chmod +x container_rule/code/serve

!ls -latr container_rule/code/serve

### Create ECR repository

In [ ]:
%%sh

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

echo "completed"

### Print out created ECR

In [ ]:
!aws ecr describe-repositories --repository-names "$algorithm_name"

### Build the Docker Container


Before we start, run the below command to take a look at the Dockerfile.

In [ ]:
!cat container_rule/Dockerfile

### The below code will login to ECR and then build the docker image and push to ECR.

In [ ]:
%%sh

cd container_rule
pwd

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region $region --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t $algorithm_name .
docker tag $algorithm_name $fullname

docker push $fullname

### Print out docker image


In [ ]:
!docker images $algorithm_name

# Part 2: Using your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train models and use the model for hosting or batch transforms. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [ ]:
import time
import boto3
import sagemaker as sage

# add role being used by notebook
role ="arn:aws:iam::111016121260:role/service-role/SC-111016121260-pp-6qlynp56-SageMakerExecutionRole-1KR1D7OKE5RN0"

## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.

### Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [ ]:
sample_count = 100

image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)
#model.Model
toc = time.time()
model = sage.model.Model(model_data="",image=image,role=role)
model.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge",wait=False)
tic = time.time()

timer = tic - toc
print("")
print(f"Time took to complete: {timer} seconds or {timer/60} minutes")

print(model.name)

#transformer.Transformer
toc = time.time()
transform = sage.transformer.Transformer(model_name=model.name,instance_count=1,instance_type="ml.c4.xlarge", max_payload=10, max_concurrent_transforms=10, strategy='MultiRecord')
transform.transform(data="s3://sagemaker-cfn/" +  str(sample_count) + "-sample.csv", content_type='text/csv', split_type='Line',wait=True)
#model.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge", wait=False)
tic = time.time()

timer = tic - toc
print("")
print(f"Time took to complete: {timer} seconds or {timer/60} minutes")

### Send data though endpoint


Get endpoint name that was just created

In [ ]:
!aws sagemaker list-endpoints --name-contains $algorithm_name 


In [ ]:
import json

sagemaker = boto3.client('sagemaker-runtime')

def invoke_ep(event):
    
    response = sagemaker.invoke_endpoint(EndpointName="python-rule-based-1-2020-03-22-20-11-21-981", # add endpoint here
                                       ContentType='text/csv',
                                       #Body=https://sagemaker-cfn.s3.us-east-2.amazonaws.com/channels+(1).csv)
                                       Body=event)

    result = response['Body'].read().decode()
    #result = json.loads(response['Body'].read().decode())
    print(result)
    
    return result

invoke_ep(b"12.01,14.05\n14.2,12.3")

In [ ]:
!aws sagemaker list-transform-jobs --name-contains $algorithm_name --query TransformJobSummaries[*].[TransformJobName,TransformJobArn,TransformJobStatus]


In [ ]:
!aws sagemaker describe-transform-job --transform-job-name python-rule-based-1-2020-03-22-20-11-22-474
